<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/multifit_fr_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
pip install transformers

In [ ]:
pip install git+https://github.com/n-waves/multifit

In [ ]:
pip install sacremoses

In [ ]:
pip install sentencepiece

In [ ]:
from zipfile import ZipFile 
path = F"/content/gdrive/My Drive/multilingual toxic/" 
with ZipFile(path+'toxic.zip', 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!')
import pandas as pd
def load_data():
    trn=pd.read_csv('jigsaw-toxic-comment-train.csv',usecols=['toxic','comment_text'])
    val=pd.read_csv('validation.csv',usecols=['toxic','comment_text','lang'])
    tst=pd.read_csv('test.csv',usecols=['lang','content'])  
    sub=pd.read_csv('submission (1).csv')  
    return trn,tst,val,sub
train,test,valid,sub=load_data()
test['toxic']=sub['toxic'].round()

In [ ]:
print(train.shape,test.shape,valid.shape,sub.shape)

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.layers import *
from fastai.text import *
import multifit
exp = multifit.from_pretrained("es_multifit_paper_version")
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
fa_config =  exp.pretrain_lm.tokenizer.get_fastai_config(add_open_file_processor=True)

In [ ]:
es=test.loc[test['lang']=='fr']
es['id']=es.index
es=es.reset_index(drop=True)
es.head()

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
es['toxic']=es['toxic'].astype(np.long)
ln=es.shape[0]//5
trn=es.loc[ln:]
val=es.loc[:ln]
print(trn.shape,val.shape)

In [ ]:
with strategy.scope():
    exp = multifit.from_pretrained("fr_multifit_paper_version")
    fa_config =  exp.pretrain_lm.tokenizer.get_fastai_config(add_open_file_processor=True)
    data_lm = (TextList.from_df(es, **fa_config)
            .split_by_rand_pct(0.1)
            .label_for_lm()           
            .databunch(bs=32))
    learn = exp.finetune_lm.get_learner(data_lm)  
    # learn is a preconfigured fastai learner with a pretrained model loaded
data_lm.show_batch()

In [ ]:
learn.fit_one_cycle(10,max_lr=1e-4)

In [ ]:
self=exp.finetune_lm
CLS_BEST = 'cls_best'
LM_BEST = "lm_best"
ENC_BEST = "enc_best"
experiment_path=Path(path)
self.experiment_path=experiment_path
tokenizer = self.base.tokenizer
print("Experiment", experiment_path)

self.experiment_path = experiment_path
tokenizer.save(self.experiment_path, learn=learn)
learn.to_fp32()
learn.save_encoder('/content/'+ENC_BEST)
learn.save(LM_BEST, with_opt=False)
# learn.destroy()
self.save_paramters()
print("Language model saved to", self.experiment_path)

In [ ]:
self=exp.classifier
self.base.experiment_path=Path('')
from pathlib import Path
with strategy.scope():
    exp.classifier.experiment_path=Path('')
    data_clas = TextClasDataBunch.from_df("", trn.reset_index(drop=True),val,  text_cols = 'content', label_cols = 'toxic',bs=32,vocab=data_lm.vocab)
    data_clas.vocab.itos = data_lm.vocab.itos
    l1rn = exp.classifier.get_learner(data_clas)  
# learn is a preconfigured fastai learner with a pretrained model loaded


In [ ]:
data_clas.show_batch()

In [ ]:
l1rn.fit_one_cycle(10,max_lr=1e-3)

In [ ]:
pre=l1rn.get_preds()[0][:,1].reshape(-1,1)

In [ ]:
val['toxic']=np.asarray(pre)
val.to_csv(path+'fr_1.csv')